<a href="https://colab.research.google.com/github/AlexandreYuferev/-Python-FastAPI-PostgreSQL-/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colaboratory!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:

import os
import zipfile

# Путь к ZIP-архиву в Google Drive
zip_path = '/content/drive/My Drive/archive.zip'

# Путь, куда нужно разархивировать файлы
extract_path = '/content/'

# Разархивируем
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
!ls /content/train/

'Cashew anthracnose'	    'Cassava healthy'	   'Maize streak virus'
'Cashew gumosis'	    'Cassava mosaic'	   'Tomato healthy'
'Cashew healthy'	    'Maize fall armyworm'  'Tomato leaf blight'
'Cashew leaf miner'	    'Maize grasshoper'	   'Tomato leaf curl'
'Cashew red rust'	    'Maize healthy'	   'Tomato septoria leaf spot'
'Cassava bacterial blight'  'Maize leaf beetle'    'Tomato verticulium wilt'
'Cassava brown spot'	    'Maize leaf blight'
'Cassava green mite'	    'Maize leaf spot'


In [ ]:

from PIL import Image

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

def get_unique_image_dimensions(root_dir):

    # Создаем множество для хранения уникальных размерностей
    unique_dimensions = set()

    # Итерируемся по всем папкам и файлам в указанной директории
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    width, height = img.size
                    # Добавляем размерности в множество
                    unique_dimensions.add((width, height))
            except Exception as e:
                print(f'Error with image {file_path}: {e}')

    return unique_dimensions

# Укажите путь к директории с изображениями
image_dir = '/content/test'
unique_dims = get_unique_image_dimensions(image_dir)

# Выводим уникальные размерности
for dim in unique_dims:
    print(f'Width: {dim[0]}, Height: {dim[1]}')


Width: 400, Height: 400


In [ ]:

class CustomDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.image_paths = []
        self.labels = []

        for idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path, img_name))
                self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((400, 400)),  # измените на размер ваших изображений
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Путь к папкам с обучающими и тестовыми данными
train_dir = '/content/train'
test_dir = '/content/test'

# Создаем экземпляры датасета
train_dataset = CustomDataset(root_dir=train_dir, transform=transform)
test_dataset = CustomDataset(root_dir=test_dir, transform=transform)

# Создаем DataLoader
train_loader = DataLoader(train_dataset, batch_size=300, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=300, shuffle=False)

# Пример итерации по train_loader
for images, labels in train_loader:
    print(f'Batch size: {images.size(0)}, Labels: {labels[0]}, Image dimensions: {images.size(1)}, {images.size(2)}, {images.size(3)}')
    break  # Выводим размерности для первого батча и выходим из цикла


Batch size: 300, Labels: 20, Image dimensions: 3, 400, 400


In [ ]:

size_0 = 400

size_1 = (size_0 + 2 * 1 - 5) / 1 - 1
size_1 /= 2
size_2 = (size_1 + 2 * 1 - 5) / 1 - 1
size_2 /= 2
size_3 = (size_2 + 2 * 1 - 4) / 1 - 1
size_3 /= 2
size_4 = (size_3 + 2 * 1 - 4) / 1 - 1
size_4 /= 2

print(size_1)
print(size_2)
print(size_3)
print(size_4)


198.0
97.0
47.0
22.0


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=9, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(9)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(in_channels=9, out_channels=12, kernel_size=4, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(12)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(in_channels=12, out_channels=15, kernel_size=4, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(15)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(15 * 23 * 23, 23 * 23)
        self.fc2 = nn.Linear(23 * 23, 22)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        # print(x.size())
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        # print(x.size())
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        # print(x.size())
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        # print(x.size())

        # Размерность x перед полносвязными слоями
        x = x.view(-1, 15 * 23 * 23)
        x = self.fc1(x)
        x = self.fc2(x)

        return x


In [ ]:

# Функция для обучения модели
def train_model(model, device, train_loader, criterion, optimizer, num_epochs=100):

    train_losses = []
    train_accuracy = []

    for epoch in range(num_epochs):
        model.train()  # переводим модель в режим обучения

        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        epoch_accuracy = 100 * correct / total
        train_accuracy.append(epoch_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Сохраняем модель каждые 100 эпох
        if (epoch + 1) % 100 == 0:
            torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pt")

    return train_losses, train_accuracy

# Функция для расчета процента правильных ответов
def calculate_accuracy(model, data_loader):

    model.eval()  # переводим модель в режим eval для оценки

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Функция для визуализации данных
def plot_data(train_losses, train_accuracy):

    plt.figure(figsize=(12, 5))

    # Визуализация функции потерь
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over epochs')
    plt.legend()

    # Визуализация процента правильных ответов
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracy, label='Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy over epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:

def main(train_loader):

    # Инициализация модели, оптимизатора и функции потерь
    model = Network()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Проверяем доступность GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Вычисления будут производиться на устройстве: {device}")

    # Перемещаем модель на GPU, если доступен
    model.to(device)

    # Обучение модели
    train_losses, train_accuracy = train_model(model, device, train_loader, criterion, optimizer, num_epochs=10000)

    # Расчет процента правильных ответов
    accuracy = calculate_accuracy(model, train_loader)
    print(f"Final Training Accuracy: {accuracy:.2f}%")

    # Визуализация данных
    plot_data(train_losses, train_accuracy)

if __name__ == "__main__":
    main(train_loader)


Вычисления будут производиться на устройстве: cpu


Epoch 1/10000:   0%|          | 0/68 [00:01<?, ?it/s]


KeyboardInterrupt: 